In [1]:
from functools import total_ordering
from itertools import count
from typing import List

In [2]:
@total_ordering
class Task:
    _ids = count(0) #shared id variable, unique for each task
    
    def __init__(self, release: int, duration: int, deadline: int):
        self._id = next(self._ids)
        self.release = release
        self.duration = duration
        self.deadline = deadline
        self.start_min = release
        self.start_max = deadline - duration
        
    def __eq__(self, other):
        return self.deadline == other.deadline
    
    def __ne__(self, other):
        return not (self.deadline == other.deadline)
    
    def __lt__(self, other):
        return self.deadline < other.deadline
    
    def __repr__(self):
        return f"Task ID: {self._id}, release: {self.release}, duration: {self.duration}, deadline: {self.deadline}"

class cEDF:
    def __init__(self, task_index: List, time):
        self._index = task_index
        self._time = time
        self._crit_q = self._critical_queue()
        self._red_q = self._ready_queue()
        
    def _critical_queue(self):
        crit_q = sorted(self._index, key=lambda x: x.start_max, reverse=False)
        return crit_q
    
    def _ready_queue(self):
        temp_q = []
        for item in self._index:
            if item.release <= self._time:
                temp_q.append(item)
        red_q = sorted(temp_q, key=lambda x: x.deadline, reverse=False)
        return red_q
    
    def _remove_task_id_queue(self, task_id: int, queue: List):
        for i in range(0, len(queue)):
            if queue[i]._id == task_id:
                del queue[i]
                break;
    
    def _remove_task_id_all(self, task_id: int):
        self._remove_task_id_queue(task_id, self._index)
        self._remove_task_id_queue(task_id, self._crit_q)
        self._remove_task_id_queue(task_id, self._red_q)
    
    def update_time(self, time: int):
        self._time = time
        self._red_q = self._ready_queue()
        
    def schedule(self):
        return NotImplementedError()

In [17]:
task_index = [Task(0, 5, 20), Task(2, 4, 10), Task(0, 3, 19)]
sched = cEDF(task_index, 0)

In [18]:
sched._index

[Task ID: 6, release: 0, duration: 5, deadline: 20,
 Task ID: 7, release: 2, duration: 4, deadline: 10,
 Task ID: 8, release: 0, duration: 3, deadline: 19]

In [19]:
sched._crit_q

[Task ID: 7, release: 2, duration: 4, deadline: 10,
 Task ID: 6, release: 0, duration: 5, deadline: 20,
 Task ID: 8, release: 0, duration: 3, deadline: 19]

In [20]:
sched._red_q

[Task ID: 8, release: 0, duration: 3, deadline: 19,
 Task ID: 6, release: 0, duration: 5, deadline: 20]

In [21]:
sched._remove_task_id_all(1)